In [ ]:
import random
import gym
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

class DQNAgent:
  def __init__(self, state_size, action_size, n_hidden, hidden_activation, out_activation):
        self.n_hidden = n_hidden
        self.hidden_activation = hidden_activation
        self.out_activation = out_activation
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        # reward discount rate
        self.gamma = 0.99    
        # exploration rate     
        self.epsilon = 1.0        
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.999
        self.learning_rate = 0.001
        self.min_memory_req = 1000
        self.model = self.create_nn()

  def create_nn(self):
      model = Sequential()
      model.add(Dense(n_hidden, input_dim=self.state_size, activation = hidden_activation))
      model.add(Dense(n_hidden, activation = hidden_activation))
      model.add(Dense(self.action_size, activation = out_activation))
      model.summary()
      model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
      return model

  # store transition to allow dqn to remember
  def store_transitions(self, state, action, reward, next_state, done):
      self.memory.append((state, action, reward, next_state, done))

  def pick_action(self, state):
      if np.random.random() < self.epsilon:
          return random.randrange(self.action_size)
      else:
          q_values = self.model.predict(state)
          return np.argmax(q_values[0])   # returns action
  
  # experience replay buffer
  def exp_replay(self, batch_size):
    # randomly select a batch 
    minibatch = random.sample(self.memory, batch_size)
    # initalize current state
    state = np.zeros((batch_size, self.state_size))
    next_state = np.zeros((batch_size, self.state_size))
    action, reward, done = [], [], []

    for i in range(batch_size):
        state[i] = minibatch[i][0]  
        action.append(minibatch[i][1])
        reward.append(minibatch[i][2])
        next_state[i] = minibatch[i][3]  
        done.append(minibatch[i][4])

    # current state q value
    target_q_val = self.model.predict(state) 
    # compute next state q value
    q_val_next_state = self.model.predict(next_state)    

    for i in range(batch_size):
        if done[i]:
            target_q_val[i][action[i]] = reward[i]
        else:
            target_q_val[i][action[i]] = reward[i] + self.gamma * (
                np.amax(q_val_next_state[i]))

    # train model
    self.model.fit(state, target_q_val, batch_size=batch_size,
                   epochs=1, verbose=0)
    if (self.epsilon > self.epsilon_min) and (len(self.memory) >= self.min_memory_req):
        self.epsilon *= self.epsilon_decay
  
  # update epsilon 
  def update_epsilon(self):
    if self.epsilon > self.epsilon_min and (len(self.memory) >= self.min_memory_req):
        self.epsilon *= self.epsilon_decay

# initialize the environment
env = gym.make('CartPole-v0')

# initalize parameters for DQN
state_size = env.observation_space.shape[0]
action_size = env.action_space.n 
n_hidden = 24
n_episodes = 200
done = False 
batch_size = 32
res_lst_1 = []
res_lst_2 = []
reward_tracker = []
hidden_activation = "relu"
out_activation = "linear"
replay_switch = [False, True]
for on_off in range(len(replay_switch)):
  use_replay = replay_switch[on_off]
  agent = DQNAgent(state_size, action_size, n_hidden, hidden_activation, out_activation)
  for i_episode in range(n_episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    done = False
    rewards = 0
    time_steps_c = 0 
    while not done:
        # env.render()
        action = agent.pick_action(state)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -100   
        next_state = np.reshape(next_state, [1, state_size])

        # remember
        agent.store_transitions(state, action, reward, next_state, done)
        state = next_state
        rewards += 1
        time_steps_c += 1
        # for experience replay
        if len(agent.memory) > batch_size and use_replay == True:
            # use experience replay if memory length enough for batching
            agent.exp_replay(batch_size) 
        #elif use_replay == False:
        
        elif use_replay == False:
          agent.update_epsilon()

    print('Episode {}\{}: done after {} timesteps, total rewards {}, exploration {:.2}, memory length {}'.format(i_episode, n_episodes, time_steps_c, rewards, agent.epsilon, len(agent.memory)))
    reward_tracker.append(rewards)
    if use_replay == True :
      res_lst_1.append(rewards)
    elif use_replay == False:
      res_lst_2.append(rewards)

    # stop training if agent solves environemnt for 25 sucessive episodes
    if np.mean(reward_tracker[-min(25, len(reward_tracker)):]) >= (env.spec.max_episode_steps - 5):
        print('Agent consequitively solved the game with a score => 195 in {} episodes'.format(i_episode))
        break

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 24)                120       
_________________________________________________________________
dense_1 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________
Episode 0\200: done after 38 timesteps, total rewards 38, exploration 1.0, memory length 38
Episode 1\200: done after 16 timesteps, total rewards 16, exploration 1.0, memory length 54
Episode 2\200: done after 21 timesteps, total rewards 21, exploration 1.0, memory length 75
Episode 3\200: done after 14 timesteps, total rewards 14, exploration 1.0, memory length 89
Episode 4\200: done 

KeyboardInterrupt: ignored